In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

pandas version:

In [ ]:
df=pd.read_csv("../input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv")
df

In [ ]:
df.info()

In [ ]:
df.columns

# Get CATEGORICAL data which is represented by int64:

sources: 
* https://pbpython.com/categorical-encoding.html=

* https://www.shanelynn.ie/using-pandas-dataframe-creating-editing-viewing-data-in-python/

* https://towardsdatascience.com/categorical-encoding-using-label-encoding-and-one-hot-encoder-911ef77fb5bd

In [ ]:
cat_df = df.select_dtypes(include=['int64']).copy()
cat_df = cat_df.drop(columns="ID")#delete ID from categorical data -> not useful
cat_df.columns

In [ ]:
cat_df.shape

1. ONE-HOT-ENCODE certain categorical data


* replaces i column with multiple columns that will be "hot" when rows with a certain status
* will not one-hot-encode AGE

ONE-HOT-ENCODE: "SEX","MARRIAGE","EDUCATION",

TOO MANY COLUMNS FOR "EDUCATION"

->replace certain education statuses due to too many cols ->put all other options into 4 

In [ ]:
cat_df['EDUCATION'].replace({0: 4, 5: 4, 6: 4}, inplace=True)

In [ ]:
encode_columns=['SEX','MARRIAGE','EDUCATION']
for i in encode_columns:
    cat_df=pd.get_dummies(cat_df, columns=[i])

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
cat_df.columns


ONE-HOT-ENCODE "PAY_i":

In [ ]:
unique_status = np.unique(cat_df[['PAY_0']])
print("total unique statuses:", len(unique_status))
print(unique_status)

* will get 10-11 new columns per PAY_i with one-hot-encoding because some monthes might have 0 frequency of a payment status/es

In [ ]:
monthes=['PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']
for i in monthes:
    cat_df=pd.get_dummies(cat_df, columns=[i])


BIN the AGE feature

5 groups : 21-30 , 31-40 , 40-50 , 50-60 , 60-75

sources:

https://medium.com/vickdata/four-feature-types-and-how-to-transform-them-for-machine-learning-8693e1c24e80

In [ ]:
bins = [21, 30, 40, 50, 60, 76]
group_names = ['21-30', '31-40', '41-50', '51-60', '61-76']
age_cats = pd.cut(cat_df['AGE'], bins, labels=group_names)
cat_df['age_cats'] = pd.cut(cat_df['AGE'], bins, labels=group_names)


ONE-HOT-ENCODE the age categories :

In [ ]:
cat_df=pd.get_dummies(cat_df, columns=['age_cats'])

In [ ]:
cat_df.head()

In [ ]:
cat_df.columns

In [ ]:
len(cat_df.columns)

In [ ]:
cat_df.dtypes

In [ ]:
len(cat_df.columns)

# Get NUMERICAL data which is represented by float64:

In [ ]:
num_df = df.select_dtypes(include=['float64']).copy()
num_df.columns

1. ADAPTIVE BINNING: the BILL_AMT cols


* we use the data distribution itself to decide our bin ranges

* bill_amts will be put into quantiles

source: https://towardsdatascience.com/understanding-feature-engineering-part-1-continuous-numeric-data-da4e47099a7b

In [ ]:
bills=['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4','BILL_AMT5','BILL_AMT6']
col_names=['Q_BILL_AMT1', 'Q_BILL_AMT2', 'Q_BILL_AMT3', 'Q_BILL_AMT4','Q_BILL_AMT5', 'Q_BILL_AMT6']
i=0#counter 

for col in bills:
    quantile_list = [0, 0.25, 0.5, 0.75, 1.0]
    quantile_labels = ['0-25Q', '25-50Q', '50-75Q', '75-100Q']
    num_df[col_names[i]] = pd.qcut(num_df[col],q=quantile_list,labels=quantile_labels)
    i+=1
    
num_df.columns

In [ ]:
num_df.head()

2. ADAPTIVE BINNING: the PAY_AMT cols AND LIMIT_BAL
we use the data distribution itself to decide our bin ranges

PAY_AMT(s) and LIMIT_BAL will be put into quantiles

source: https://towardsdatascience.com/understanding-feature-engineering-part-1-continuous-numeric-data-da4e47099a7b

In [ ]:
pays=['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5','PAY_AMT6','LIMIT_BAL']
col_names=['Q_PAY_AMT1', 'Q_PAY_AMT2', 'Q_PAY_AMT3','Q_PAY_AMT4','Q_PAY_AMT5','Q_PAY_AMT6','Q_LIMIT_BAL']
i=0#counter 

for col in pays:
    quantile_list = [0, 0.25, 0.5, 0.75, 1.0]
    quantile_labels = ['0-25Q', '25-50Q', '50-75Q', '75-100Q']
    num_df[col_names[i]] = pd.qcut(num_df[col],q=quantile_list,labels=quantile_labels)
    i+=1
    
num_df.columns

now the originally numerical columns are categorical columns 

ONE-HOT-ENCODE the Q_PAY_AMTs , Q_BILL_AMTs, and Q_LIM_BAL

In [ ]:
encode_columns=['Q_BILL_AMT1', 'Q_BILL_AMT2','Q_BILL_AMT3', 'Q_BILL_AMT4', 'Q_BILL_AMT5', 'Q_BILL_AMT6','Q_PAY_AMT1', 'Q_PAY_AMT2', 'Q_PAY_AMT3','Q_PAY_AMT4','Q_PAY_AMT5','Q_PAY_AMT6','Q_LIMIT_BAL']
for i in encode_columns:
    num_df=pd.get_dummies(num_df, columns=[i])

In [ ]:
num_df.head()

In [ ]:
num_df.columns

In [ ]:
len(num_df.columns)

NEW COLUMN: create column that indicates if tuple has payment status >1 in first month and last month


* make loop to go thru each PAY_0
* make col with 0 or 1 that will indicate if condtion in true 
* add to num_df 


source:https://stackoverflow.com/questions/32984462/setting-1-or-0-to-new-pandas-column-conditionally

In [ ]:
num_df['late_payer']=df['PAY_0'].apply(lambda x: 1 if x > 1 else 0)

num_df['late_payer'].head()

NEW COLUMN: create column that indicates if tuple has payed more than BILL_AMT (meaning they have a negative balance)

In [ ]:
bill_mons=['BILL_AMT1','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5','BILL_AMT6']
cols=['OVER_BILL_AMT1','OVER_BILL_AMT2','OVER_BILL_AMT3','OVER_BILL_AMT4','OVER_BILL_AMT5','OVER_BILL_AMT6']
i=0#counter

for mon in bill_mons:
    num_df[cols[i]]=df[mon].apply(lambda x: 1 if x < 0 else 0)
    i+=1
    
num_df['OVER_BILL_AMT1'].head()    

CONCAT ALL DATAFRAMES MADE:

In [ ]:
data = pd.concat([cat_df, num_df], axis=1)

In [ ]:
data.to_csv('mycsvfile.csv',index=False)

In [ ]:
data.head()

In [ ]:
len(data.columns)

153 columns in all (with target col): next step is creating model with this df 

In [ ]:
data.dtypes

# MAKE copies of dataframe w/o target column

In [ ]:
data_=data[data.columns[~data.columns.isin(['default.payment.next.month'])]]#already does not have ID

In [ ]:
target=data['default.payment.next.month']

# LOGISTIC REGRESSION: 

source: https://www.kaggle.com/mnassrib/titanic-logistic-regression-with-python#4.-Logistic-Regression-and-Results

* this is exact code from the section called "Logistic Regression and Results" 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score 
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, auc, log_loss
from sklearn import preprocessing
import matplotlib.pyplot as plt 

# create X (features) and y (response)
X = data_
y = target

# use train/test split with different random_state values
# we can change the random_state values that changes the accuracy scores
# the scores change a lot, this is why testing scores is a high-variance estimate
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# check classification scores of logistic regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]
[fpr, tpr, thr] = roc_curve(y_test, y_pred_proba)
print('Train/Test split results:')
print(logreg.__class__.__name__+" accuracy is %2.3f" % accuracy_score(y_test, y_pred))
print(logreg.__class__.__name__+" log_loss is %2.3f" % log_loss(y_test, y_pred_proba))
print(logreg.__class__.__name__+" auc is %2.3f" % auc(fpr, tpr))

idx = np.min(np.where(tpr > 0.95)) # index of the first threshold for which the sensibility > 0.95

#plot
plt.figure()
plt.plot(fpr, tpr, color='coral', label='ROC curve (area = %0.3f)' % auc(fpr, tpr))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot([0,fpr[idx]], [tpr[idx],tpr[idx]], 'k--', color='blue')
plt.plot([fpr[idx],fpr[idx]], [0,tpr[idx]], 'k--', color='blue')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1 - specificity)', fontsize=14)
plt.ylabel('True Positive Rate (recall)', fontsize=14)
plt.title('Receiver operating characteristic (ROC) curve')
plt.legend(loc="lower right")
plt.show()

print("Using a threshold of %.3f " % thr[idx] + "guarantees a sensitivity of %.3f " % tpr[idx] +  
      "and a specificity of %.3f" % (1-fpr[idx]) + 
      ", i.e. a false positive rate of %.2f%%." % (np.array(fpr[idx])*100))

# precision-recall curve

source: https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/

In [ ]:
# precision-recall curve and f1
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc
from matplotlib import pyplot

# split into train/test sets
# create X (features) and y (response)
X = data_
y = target

# use train/test split with different random_state values
# we can change the random_state values that changes the accuracy scores
# the scores change a lot, this is why testing scores is a high-variance estimate
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# fit a model
model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)
# predict probabilities
lr_probs = model.predict_proba(X_test)
# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]
# predict class values
yhat = model.predict(X_test)
lr_precision, lr_recall, _ = precision_recall_curve(y_test, lr_probs)
lr_f1, lr_auc = f1_score(y_test, yhat), auc(lr_recall, lr_precision)
# summarize scores
print('Logistic: f1=%.3f AUPR=%.3f' % (lr_f1, lr_auc))
# plot the precision-recall curves
no_skill = len(y_test[y_test==1]) / len(y_test)
pyplot.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No Skill')
pyplot.plot(lr_recall, lr_precision, marker='.', label='Logistic')
# axis labels
pyplot.xlabel('Recall')
pyplot.ylabel('Precision')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()

false positive: lots of defaults are being reported when they don't exist actually 

another source: 

https://towardsdatascience.com/logistic-regression-detailed-overview-46c4da4303bc#:~:text=Data%20is%20fit%20into%20linear,the%20target%20categorical%20dependent%20variable.&text=To%20predict%20which%20class%20a,probability%20is%20classified%20into%20classes.

# h2o work
source: https://h2oai.github.io/tutorials/introduction-to-machine-learning-with-h2o-part-1/#2

# GLM: H2O Generalized Linear Model

In [ ]:
import h2o
h2o.init()

In [ ]:
#Import H2O and other libraries that will be used in this tutorial 
import matplotlib as plt
%matplotlib inline

#Import the Estimators
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator

#Import h2o grid search 
import h2o.grid 
from h2o.grid.grid_search import H2OGridSearch

In [ ]:
clus=h2o.H2OFrame(data_)
clus.head()

In [ ]:
import os

startup  = '/home/h2o/bin/aquarium_startup'
shutdown = '/home/h2o/bin/aquarium_stop'

if os.path.exists(startup):
    os.system(startup)
    local_url = 'http://localhost:54321/h2o'
    aquarium = True
else:
    local_url = 'http://localhost:54321'
    aquarium = False

In [ ]:
h2o.init(url=local_url)

need target column

In [ ]:
orig_data=h2o.import_file("../input/default-of-credit-card-clients-dataset/UCI_Credit_Card.csv")

In [ ]:
target=orig_data['default.payment.next.month']

In [ ]:
target.head()

In [ ]:
clus['default.payment.next.month']=orig_data['default.payment.next.month']

In [ ]:
train, test = clus.split_frame([0.8], seed=42)

In [ ]:
print("train:%d test:%d" % (train.nrows,test.nrows))

In [ ]:
y = 'default.payment.next.month'

ignore = ["default.payment.next.month"] 

x = list(set(train.names) - set(ignore))

print(x)


In [ ]:
train.names

In [ ]:
glm = H2OGeneralizedLinearEstimator(family = "binomial", seed=42, model_id = 'default_glm')

In [ ]:
%time glm.train(x = x, y = y, training_frame = train)

In [ ]:
glm

In [ ]:
glm.plot(metric='negative_log_likelihood')

In [ ]:
glm.varimp_plot()

In [ ]:
glm.predict(test).head(10)

save default performance to compare later on: 

In [ ]:
default_glm_perf=glm.model_performance(test)

In [ ]:
print("AUC: ",default_glm_perf.auc())

In [ ]:
print("ACCURACY: ",default_glm_perf.accuracy())

[threshold, accuracy]

In [ ]:
print("F1 score: ",default_glm_perf.F1())

[threshold, f1_score]

# RANDOM FOREST WITH h2o

In [ ]:
from h2o.model.confusion_matrix import ConfusionMatrix
rf = H2ORandomForestEstimator (seed=42, model_id='default_rf')
%time rf.train(x=x, y=y, training_frame=train)

In [ ]:
rf

In [ ]:
rf.plot(metric='AUTO')

In [ ]:
rf.varimp_plot(20)

# GBM:H2OGradientBoostingEstimator

In [ ]:
clus['default.payment.next.month'] = clus['default.payment.next.month'].asfactor()    

train,test = clus.split_frame([0.7], seed=42)

y = 'default.payment.next.month'

x = list(set(train.names))

gbm= H2OGradientBoostingEstimator(seed=42, model_id='default_gbm')
%time gbm.train(x=x, y=y, training_frame=train)

In [ ]:
gbm

In [ ]:
default_gbm_perf=gbm.model_performance(test)

In [ ]:
default_gbm_perf.accuracy()

In [ ]:
gbm.varimp_plot(20)